In [1]:
# Check the data
import pandas as pd
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
stations = os.listdir("/home/sebas/data/air_qual_aemet/")
stations = pd.Series(stations).apply(lambda e: e.split(".csv")[0]).values

In [3]:
seq_length=168

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
all_dfs=np.array([])
for station in stations:
    df = pd.read_csv(f"/home/sebas/data/air_qual_aemet/{station}.csv",sep=";")
    all_dfs = np.concatenate([all_dfs,df["SPA.NO2"].values])
scaler.fit(all_dfs.reshape(-1, 1))

MinMaxScaler()

In [5]:
X_train=[]
y_train=[]
for station in stations:
    df = pd.read_csv(f"/home/sebas/data/air_qual_aemet/{station}.csv",sep=";")
    # Limiting Date
    df = df[df["DATE"]<"2020-01-01"]

    tdf = df[["DATE","SPA.NO2"]].copy()
    tdf.columns = ["DATE","NO2"]
    tdf["NO2"]=np.log1p(tdf["NO2"])

    from statsmodels.tsa.seasonal import seasonal_decompose
    stl = seasonal_decompose(tdf["NO2"], model="additive",period=24)
    tdf["NO2"]=tdf["NO2"]-stl.seasonal
    tdf["trend"]=stl.trend

    horizon=13

    tdf["trend_norm"]=tdf["trend"].shift(horizon)

    # We remove the trend and keep past values
    tdf["NO2"]=tdf["NO2"]-tdf["trend_norm"]
    for h in np.arange(0,horizon+seq_length+1):
        if h>horizon:
            tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()
            tdf[f"NO2 - {h}"]=scaler.transform()

    tdf=tdf.dropna()

    cols = tdf.columns[tdf.columns.str.contains("NO2 -")]
    X = tdf[cols].copy()
    y = tdf[["NO2"]].copy()
    
    TRAIN_SPLIT = tdf[tdf["DATE"]>"2018"].index.values[0]

    X_train.append(X[X.index<=TRAIN_SPLIT].copy())
    y_train.append(y[X.index<=TRAIN_SPLIT].copy())
X_train=pd.concat(X_train)
y_train=pd.concat(y_train)

TypeError: MinMaxScaler.transform() missing 1 required positional argument: 'X'

In [5]:
X.shape

(56772, 168)

In [6]:
X.head()

,NO2 - 14,NO2 - 15,NO2 - 16,NO2 - 17,NO2 - 18,NO2 - 19,NO2 - 20,NO2 - 21,NO2 - 22,NO2 - 23,...,NO2 - 172,NO2 - 173,NO2 - 174,NO2 - 175,NO2 - 176,NO2 - 177,NO2 - 178,NO2 - 179,NO2 - 180,NO2 - 181
206,-2.996216,-2.799638,-2.690678,-2.666934,-3.028388,-3.489520,-3.971476,-3.925207,-3.004463,-2.872405,...,-3.400338,-3.655107,-3.536014,-3.722969,-3.591644,-3.319321,-3.109680,-3.173329,-3.347363,-3.248272
207,-3.175049,-2.946764,-2.750186,-2.641226,-2.617483,-2.978936,-3.440068,-3.922024,-3.875756,-2.955011,...,-3.089770,-3.350886,-3.605656,-3.486562,-3.673517,-3.542193,-3.269869,-3.060228,-3.123877,-3.297912
208,-3.346077,-3.140174,-2.911890,-2.715312,-2.606352,-2.582608,-2.944062,-3.405194,-3.887149,-3.840881,...,-2.858500,-3.054895,-3.316011,-3.570781,-3.451688,-3.638642,-3.507318,-3.234995,-3.025354,-3.089003
209,-3.150227,-3.319345,-3.113443,-2.885158,-2.688581,-2.579620,-2.555877,-2.917330,-3.378462,-3.860418,...,-2.873764,-2.831769,-3.028164,-3.289280,-3.544050,-3.424956,-3.611911,-3.480587,-3.208263,-2.998622
210,-3.084282,-3.159633,-3.328752,-3.122849,-2.894565,-2.697987,-2.589026,-2.565283,-2.926736,-3.387868,...,-2.772771,-2.883171,-2.841175,-3.037570,-3.298686,-3.553456,-3.434362,-3.621317,-3.489993,-3.217669


In [7]:
len(X_train)

942360

In [8]:
station="28079004"
df = pd.read_csv(f"/home/sebas/data/air_qual_aemet/{station}.csv",sep=";")
# Limiting Date
df = df[df["DATE"]<"2020-01-01"]

tdf = df[["DATE","SPA.NO2"]].copy()
tdf.columns = ["DATE","NO2"]
tdf["NO2"]=np.log1p(tdf["NO2"])

from statsmodels.tsa.seasonal import seasonal_decompose
stl = seasonal_decompose(tdf["NO2"], model="additive",period=24)
tdf["NO2"]=tdf["NO2"]-stl.seasonal
tdf["trend"]=stl.trend

horizon=13

tdf["trend_norm"]=tdf["trend"].shift(horizon)

# We remove the trend and keep past values
tdf["NO2"]=tdf["NO2"]-tdf["trend_norm"]
for h in np.arange(0,horizon+seq_length+1):
    if h>horizon:
        tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()

tdf=tdf.dropna()

cols = tdf.columns[tdf.columns.str.contains("NO2 -")]
X = tdf[cols].copy()
y = tdf[["NO2"]].copy()

TRAIN_SPLIT = tdf[tdf["DATE"]>"2018"].index.values[0]

X_test=X[X.index>TRAIN_SPLIT].copy()
y_test = y[X.index>TRAIN_SPLIT].copy()

/tmp/ipykernel_2375/2347121095.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()
/tmp/ipykernel_2375/2347121095.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()
/tmp/ipykernel_2375/2347121095.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_2375/2347121095.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()
/tmp/ipykernel_2375/2347121095.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()
/tmp/ipykernel_2375/2347121095.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

In [9]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train = torch.FloatTensor(X_train.values).to(device)
X_test = torch.FloatTensor(X_test.values).to(device)
y_train = torch.FloatTensor(y_train.values).to(device)
y_test= torch.FloatTensor(y_test.values).to(device)

In [10]:
# https://github.com/harryliew/COCOB-optimizer/blob/master/cocob_bp.py

import torch.optim as optim
import torch

###########################################################################
# Training Deep Networks without Learning Rates Through Coin Betting
# Paper: https://arxiv.org/abs/1705.07795
#
# NOTE: This optimizer is hardcoded to run on GPU, needs to be parametrized
###########################################################################

class COCOBBackprop(optim.Optimizer):
    
    def __init__(self, params, alpha=100, epsilon=1e-8):
        
        self._alpha = alpha
        self.epsilon = epsilon
        defaults = dict(alpha=alpha, epsilon=epsilon)
        super(COCOBBackprop, self).__init__(params, defaults)
        
    def step(self, closure=None):
        
        loss = None
        
        if closure is not None:
            loss = closure()
            
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
        
                grad = p.grad.data
                state = self.state[p]
                
                if len(state) == 0:
                    state['gradients_sum'] = torch.zeros_like(p.data).cuda().float()
                    state['grad_norm_sum'] = torch.zeros_like(p.data).cuda().float()
                    state['L'] = self.epsilon * torch.ones_like(p.data).cuda().float()
                    state['tilde_w'] = torch.zeros_like(p.data).cuda().float()
                    state['reward'] = torch.zeros_like(p.data).cuda().float()
                    
                gradients_sum = state['gradients_sum']
                grad_norm_sum = state['grad_norm_sum']
                tilde_w = state['tilde_w']
                L = state['L']
                reward = state['reward']
                
                zero = torch.cuda.FloatTensor([0.])
                
                L_update = torch.max(L, torch.abs(grad))
                gradients_sum_update = gradients_sum + grad
                grad_norm_sum_update = grad_norm_sum + torch.abs(grad)
                reward_update = torch.max(reward - grad * tilde_w, zero)
                new_w = -gradients_sum_update/(L_update * (torch.max(grad_norm_sum_update + L_update, self._alpha * L_update)))*(reward_update + L_update)
                p.data = p.data - tilde_w + new_w
                tilde_w_update = new_w
                
                state['gradients_sum'] = gradients_sum_update
                state['grad_norm_sum'] = grad_norm_sum_update
                state['L'] = L_update
                state['tilde_w'] = tilde_w_update
                state['reward'] = reward_update

        return loss


In [11]:
from sklearn.metrics import mean_squared_error

In [16]:
import lightgbm as lgb

lgbmodel = lgb.LGBMRegressor(n_estimators=1600,
                                    random_state=2020,
                                  max_depth=6)


lgbmodel.fit(X_train.cpu(),y_train.cpu())
predictions = lgbmodel.predict(X_test.cpu())
np.sqrt(mean_squared_error(predictions,y_test.cpu()))

/home/sebas/miniconda3/envs/phd/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.4585537530891118

In [58]:
from sklearn.metrics import mean_squared_error

class GRUModel(nn.Module):
    def __init__(self):
        super(GRUModel, self).__init__()
        
        self.input_size = 1
        self.output_size=1
        self.hidden_dim=64
        self.n_layers=2

        # GRU layers
        self.gru = nn.GRU(
            self.input_size, self.hidden_dim, self.n_layers, batch_first=True
        )

        # Fully connected layer
        self.fc = nn.Linear(self.hidden_dim, self.output_size)

    def forward(self, x):
        self.gru.flatten_parameters()
        
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

net = GRUModel() 
net.to(device)

criterion = nn.MSELoss(reduction='sum')
optimizer = COCOBBackprop(net.parameters())
optimizer = optim.Adam(net.parameters(),lr=0.01,weight_decay=0.001)
batch_size = 3000
epochs = 6

train = TensorDataset(X_train, y_train)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

for j in range(epochs):
    m_loss = []
    for i,batch in enumerate(train_loader):
        train_features, train_labels = batch
        y_hat = net.forward(train_features.reshape([len(train_features), -1, 1])).to(device)
        loss = criterion(y_hat, train_labels)
        m_loss.append(loss.detach().cpu().numpy())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    test = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)
    predictions=np.array([])
    for i,batch in enumerate(test_loader):
        test_b = batch[0]
        predictions = np.concatenate([predictions,net(test_b.reshape([len(test_b), seq_length,1])).cpu().reshape(-1).detach().numpy()])
    #loss_arr.append(np.mean(m_loss))
    #val_loss_arr.append(np.sqrt(mean_squared_error(predictions,y_test.cpu())))
    print(f'Epoch: {j} Loss: {np.mean(m_loss)} Val Loss: {np.sqrt(mean_squared_error(predictions,y_test.cpu()))}')

Epoch: 0 Loss: 882.078369140625 Val Loss: 0.5082942461847594
Epoch: 1 Loss: 817.6134033203125 Val Loss: 0.5370150952155733
Epoch: 2 Loss: 769.1175537109375 Val Loss: 0.5441856216771758


KeyboardInterrupt: 

In [42]:
from torch.autograd import Variable

class Net(nn.Module):
  def __init__(self):
    self.input_size = 1
    self.output_size=1
    self.hidden_dim=64
    self.n_layers=2

    super(Net,self).__init__() 
    self.lstm = nn.LSTM(self.input_size, self.hidden_dim, self.n_layers, batch_first=True)
    self.fc = nn.Linear(self.hidden_dim*seq_length, self.output_size)


  def forward(self,x):
    self.lstm.flatten_parameters()
    h_0 = Variable(torch.zeros(self.n_layers, x.size(0), self.hidden_dim)).to(device)
    c_0 = Variable(torch.zeros(self.n_layers, x.size(0), self.hidden_dim)).to(device)
    
    out, (hidden,_) = self.lstm(x,(h_0.detach(), c_0.detach()))
    out=out.reshape([len(x),self.hidden_dim*seq_length])
    # get final output 
    output = self.fc(out)
    
    return output

net = Net() 
net.to(device)

criterion = nn.MSELoss(reduction='sum')
#optimizer = COCOBBackprop(net.parameters())
optimizer = optim.Adam(net.parameters(),lr=0.00001,weight_decay=0.000001)
batch_size = 2000
epochs = 6

train = TensorDataset(X_train, y_train)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

for j in range(epochs):
    m_loss = []
    for i,batch in enumerate(train_loader):
        train_features, train_labels = batch
        y_hat = net.forward(train_features.reshape([len(train_features), seq_length ,1])).to(device)
        loss = criterion(y_hat, train_labels)
        m_loss.append(loss.detach().cpu().numpy())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    test = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)
    predictions=np.array([])
    for i,batch in enumerate(test_loader):
        test_b = batch[0]
        predictions = np.concatenate([predictions,net(test_b.reshape([len(test_b), seq_length,1])).cpu().reshape(-1).detach().numpy()])
    #loss_arr.append(np.mean(m_loss))
    #val_loss_arr.append(np.sqrt(mean_squared_error(predictions,y_test.cpu())))
    print(f'Epoch: {j} Loss: {np.mean(m_loss)} Val Loss: {np.sqrt(mean_squared_error(predictions,y_test.cpu()))}')

Epoch: 0 Loss: 586.9235229492188 Val Loss: 0.4978141213669613
Epoch: 1 Loss: 566.3613891601562 Val Loss: 0.5075631439468482


KeyboardInterrupt: 

In [68]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 

n_channels = 64
k_size = 3
n_dil = 2

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__() 
    self.features = nn.Sequential(
        nn.Conv1d(in_channels=1, out_channels=n_channels, kernel_size=k_size, padding=1),
        nn.MaxPool1d(2),
        nn.ReLU(),
    )
    
    self.regressor = nn.Sequential(
        nn.LayerNorm(n_channels*84),
        nn.Linear(n_channels*84,200),
        nn.ReLU(),
        nn.LayerNorm(200),
        nn.Linear(200,1)
    )

  def forward(self,x):
    x = self.features(x)
    #print(x.shape)
    x = x.view(-1,n_channels*x.shape[2])
    return self.regressor(x)

net = Net() 
net.to(device)
criterion = nn.MSELoss(reduction='sum')
optimizer = COCOBBackprop(net.parameters())
lr=0.01
wd=lr/100
optimizer = optim.Adam(net.parameters(),lr=lr,weight_decay=wd)

loss_arr = []
val_loss_arr = []
epochs = 10

X_train_gf = X_train.reshape(len(X_train),1,len(X_train[0]))
train = TensorDataset(X_train_gf, y_train)
train_loader = DataLoader(train, batch_size=2048, shuffle=True)

for j in range(epochs):
    m_loss = []
    for i,batch in enumerate(train_loader):
      train_features, train_labels = batch
      y_hat = net.forward(train_features)
      loss = criterion(y_hat, train_labels)
      m_loss.append(loss.cpu().detach().numpy())

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    loss_arr.append(np.mean(m_loss))

    X_test_gf = X_test.reshape(len(X_test),1,len(X_test[0]))
    predictions = net(X_test_gf).cpu().reshape(-1).detach().numpy()
    val_loss_arr.append(np.sqrt(mean_squared_error(predictions,y_test.cpu())))

    print(f'Epoch: {j} Loss: {np.mean(m_loss)} Val Loss: {np.sqrt(mean_squared_error(predictions,y_test.cpu()))}')


Epoch: 0 Loss: 1011.3659057617188 Val Loss: 0.47193238139152527
Epoch: 1 Loss: 543.7345581054688 Val Loss: 0.46646928787231445
Epoch: 2 Loss: 541.357177734375 Val Loss: 0.4671001434326172
Epoch: 3 Loss: 540.1487426757812 Val Loss: 0.466011106967926


KeyboardInterrupt: 

In [15]:
from sklearn.metrics import mean_squared_error

n_channels = 32
k_size = 3
n_dil = 2 

class GRUModel(nn.Module):
    def __init__(self):
        super(GRUModel, self).__init__()
        
        self.features = nn.Sequential(
           nn.Conv1d(in_channels=1, out_channels=n_channels, kernel_size=k_size, padding=1),
           nn.MaxPool1d(2),
        )
        
        # GRU layers
        self.output_size=1
        self.hidden_dim=64
        self.n_layers=1
        
        self.gru = nn.GRU(
            n_channels, self.hidden_dim, self.n_layers, batch_first=True
        )

        # Fully connected layer
        self.fc = nn.Linear(self.hidden_dim, self.output_size)

    def forward(self, x):
        self.gru.flatten_parameters()
        
        feat_in = self.features(x)
        #print(f"feat_in: {feat_in.shape}")
        feat_in = feat_in.reshape([len(feat_in), 84 ,32])
        
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(feat_in, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

net = GRUModel() 
net.to(device)

criterion = nn.MSELoss(reduction='sum')
optimizer = COCOBBackprop(net.parameters())
#optimizer = optim.Adam(net.parameters(),lr=0.01,weight_decay=0.001)
batch_size = 256
epochs = 6

X_train_gf = X_train.reshape(len(X_train),1,len(X_train[0]))
train = TensorDataset(X_train_gf, y_train)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

for j in range(epochs):
    m_loss = []
    for i,batch in enumerate(train_loader):
      train_features, train_labels = batch
      y_hat = net.forward(train_features)
      loss = criterion(y_hat, train_labels)
      m_loss.append(loss.cpu().detach().numpy())

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      

    X_test_gf = X_test.reshape(len(X_test),1,len(X_test[0]))
    predictions = net(X_test_gf).cpu().reshape(-1).detach().numpy()
    #val_loss_arr.append(np.sqrt(mean_squared_error(predictions,y_test.cpu())))

    print(f'Epoch: {j} Loss: {np.mean(m_loss)} Val Loss: {np.sqrt(mean_squared_error(predictions,y_test.cpu()))}')


Epoch: 0 Loss: 68.74227905273438 Val Loss: 0.46806034445762634
Epoch: 1 Loss: 66.00784301757812 Val Loss: 0.4643477201461792
Epoch: 2 Loss: 64.43340301513672 Val Loss: 0.4651913344860077
Epoch: 3 Loss: 63.219181060791016 Val Loss: 0.4651828706264496


KeyboardInterrupt: 

# Test solo station

In [36]:
X_trains=[]
y_trains=[]
station="28079004"

df = pd.read_csv(f"/home/sebas/data/air_qual_aemet/{station}.csv",sep=";")
# Limiting Date
df = df[df["DATE"]<"2020-01-01"]

tdf = df[["DATE","SPA.NO2"]].copy()
tdf.columns = ["DATE","NO2"]
tdf["NO2"]=np.log1p(tdf["NO2"])

from statsmodels.tsa.seasonal import seasonal_decompose
stl = seasonal_decompose(tdf["NO2"], model="additive",period=24)
tdf["NO2"]=tdf["NO2"]-stl.seasonal
tdf["trend"]=stl.trend

horizon=13

tdf["trend_norm"]=tdf["trend"].shift(horizon)

# We remove the trend and keep past values
tdf["NO2"]=tdf["NO2"]-tdf["trend_norm"]
for h in np.arange(0,horizon+seq_length+1):
    if h>horizon:
        tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()

tdf=tdf.dropna()

cols = tdf.columns[tdf.columns.str.contains("NO2 -")]
X = tdf[cols].copy()
y = tdf[["NO2"]].copy()

TRAIN_SPLIT = tdf[tdf["DATE"]>"2018"].index.values[0]

X_trains=X[X.index<=TRAIN_SPLIT].copy()
y_trains=y[X.index<=TRAIN_SPLIT].copy()

/tmp/ipykernel_2375/2698257401.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()
/tmp/ipykernel_2375/2698257401.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()
/tmp/ipykernel_2375/2698257401.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_2375/2698257401.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()
/tmp/ipykernel_2375/2698257401.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()
/tmp/ipykernel_2375/2698257401.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_2375/2698257401.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()
/tmp/ipykernel_2375/2698257401.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdf[f"NO2 - {h}"]=(tdf["NO2"].shift(h)-tdf["trend_norm"]).copy()
/tmp/ipykernel_2375/2698257401.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

In [37]:
import lightgbm as lgb

lgbmodel = lgb.LGBMRegressor(n_estimators=1600,
                                    random_state=2020,
                                  max_depth=6)


lgbmodel.fit(X_trains,y_trains)
predictions = lgbmodel.predict(X_test.cpu())
np.sqrt(mean_squared_error(predictions,y_test.cpu()))
#0.47

0.4744700562762055

In [34]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 

n_channels = 64
k_size = 3
n_dil = 2

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__() 
    self.features = nn.Sequential(
        nn.Conv1d(in_channels=1, out_channels=n_channels, kernel_size=k_size, padding=1),
        nn.MaxPool1d(2),
        nn.ReLU(),
    )
    
    self.regressor = nn.Sequential(
        nn.LayerNorm(n_channels*84),
        nn.Linear(n_channels*84,200),
        nn.ReLU(),
        nn.LayerNorm(200),
        nn.Linear(200,1)
    )

  def forward(self,x):
    x = self.features(x)
    #print(x.shape)
    x = x.view(-1,n_channels*x.shape[2])
    return self.regressor(x)

net = Net() 
net.to(device)
criterion = nn.MSELoss(reduction='sum')
optimizer = COCOBBackprop(net.parameters())

loss_arr = []
val_loss_arr = []
epochs = 30


X_trains = torch.FloatTensor(X_trains.values).to(device)
y_trains = torch.FloatTensor(y_trains.values).to(device)
X_trains_gf = X_trains.reshape(len(X_trains),1,len(X_trains[0]))
train = TensorDataset(X_trains_gf, y_trains)
train_loader = DataLoader(train, batch_size=4096*2, shuffle=True)

for j in range(epochs):
    m_loss = []
    for i,batch in enumerate(train_loader):
      train_features, train_labels = batch
      y_hat = net.forward(train_features)
      loss = criterion(y_hat, train_labels)
      m_loss.append(loss.cpu().detach().numpy())

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    loss_arr.append(np.mean(m_loss))

    X_test_gf = X_test.reshape(len(X_test),1,len(X_test[0]))
    predictions = net(X_test_gf).cpu().reshape(-1).detach().numpy()
    val_loss_arr.append(np.sqrt(mean_squared_error(predictions,y_test.cpu())))

    print(f'Epoch: {j} Loss: {np.mean(m_loss)} Val Loss: {np.sqrt(mean_squared_error(predictions,y_test.cpu()))}')


Epoch: 0 Loss: 47600.953125 Val Loss: 0.5063426494598389
Epoch: 1 Loss: 1808.752685546875 Val Loss: 0.49405044317245483
Epoch: 2 Loss: 1755.691162109375 Val Loss: 0.4936702251434326
Epoch: 3 Loss: 1753.1826171875 Val Loss: 0.4932498633861542
Epoch: 4 Loss: 1750.383056640625 Val Loss: 0.49275586009025574
Epoch: 5 Loss: 1747.297119140625 Val Loss: 0.4921426475048065
Epoch: 6 Loss: 1743.7203369140625 Val Loss: 0.4914079010486603
Epoch: 7 Loss: 1738.0550537109375 Val Loss: 0.4903293550014496
Epoch: 8 Loss: 1729.956787109375 Val Loss: 0.4887824058532715
Epoch: 9 Loss: 1718.042724609375 Val Loss: 0.4861428737640381
Epoch: 10 Loss: 1699.183349609375 Val Loss: 0.4819730818271637
Epoch: 11 Loss: 1671.3616943359375 Val Loss: 0.47864410281181335
Epoch: 12 Loss: 1704.7587890625 Val Loss: 0.47812405228614807
Epoch: 13 Loss: 1646.2503662109375 Val Loss: 0.47287416458129883
Epoch: 14 Loss: 1623.87109375 Val Loss: 0.47067248821258545
Epoch: 15 Loss: 1622.4967041015625 Val Loss: 0.4708791673183441
Epoc